In [2]:
from ultralytics import YOLO
import cv2
import imageio

In [4]:
### copied fire_detection class from fire.py

class fire_detection():
    def __init__(self,model_path,conf=0.85):
        self.model = YOLO(model_path,verbose=False)
        self.confidence = conf

    def process(self,img):
        bb_boxes=[]
        result=self.model(img,verbose=False)

        for box in result[0].boxes:
            if(float(box.conf[0])>self.confidence):
                bb=list(map(int,box.xyxy[0]))
                bb_boxes.append(bb)

        if(len(bb_boxes)):
            found=True
        else:
            found=False
        return (found,bb_boxes)

In [5]:
## make sure you give the full path to the model
## as of now model is trained to less data, so pass lesser conf

model=fire_detection(model_path="D:\DA\HackForge\Data-Dazzlers\det_models\\fire(1).pt",conf=0.35)

In [5]:
## loading the sample image
fire_img=cv2.imread("fire.png")

In [6]:
results=model.process(fire_img)

In [7]:
results

(True, [[31, 106, 82, 189], [205, 127, 249, 224], [128, 172, 168, 202]])

In [6]:
## testing the model on a smaple video and writing the output into a video file

cap=cv2.VideoCapture("fire_video_input.mp4")
frames=[]
ret=True
while(ret):
    ret,frame=cap.read()
    if not ret:
        break

    found,bb_box=model.process(frame)
    
    if found:
        for box in bb_box:
            x1,y1,x2,y2=box
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),2)
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frames.append(frame)

# Release video capture
cap.release()

In [7]:
gif_path = 'fire_output.gif'
imageio.mimsave(gif_path, frames, duration=0.1)